In [1]:
import pandas as pd
import yfinance as yf
import pandas_ta as ta 
import numpy as np
from backtesting import Backtest, Strategy

## Testing Bollinger Bands Long-Only Strategy with Stop-Loss On NIFTY50 Data

### Strategy Overview:

The **Bollinger Bands Long-Only Strategy** aims to take advantage of price volatility by using Bollinger Bands as a signal for potential price reversals. This strategy is designed to enter long positions when the price crosses below the lower Bollinger Band and exit the position when the price crosses above the upper Bollinger Band. Additionally, the strategy incorporates a configurable stop-loss mechanism to protect against downside risk by closing trades if the price drops by a specified percentage.

### Bollinger Bands:

- **Bollinger Bands** are volatility bands placed above and below a moving average of the asset's price.

- **Upper Band**: Calculated as the moving average plus a multiple (usually 2) of the standard deviation.

- **Lower Band**: Calculated as the moving average minus a multiple (usually 2) of the standard deviation.

### Key Features:

1\. **Long-Only Positions**:

   - The strategy is long-only, meaning it does not short the asset.

   - It buys when the price crosses below the lower Bollinger Band, indicating the asset is potentially oversold.

2\. **Exit Strategy**:

   - The strategy sells (closes the long position) when the price crosses above the upper Bollinger Band, indicating that the asset may be overbought.

3\. **Stop-Loss Protection**:

   - The strategy includes a stop-loss that is triggered if the price falls below a certain percentage from the entry price.

   - The stop-loss can be customized, and the default is set to 5% below the entry price. This ensures that losses are limited in case of a downward trend after entering the position.

### Parameters:

- **Bollinger Band Period**: 20 periods (default).

- **Bollinger Band Multiplier**: 2 (default), which defines the width of the bands based on standard deviation.

- **Stop-Loss Percentage**: Configurable (default is 5%).

### Trade Execution:

- **Buy Signal**: A buy order is executed when the price crosses below the lower Bollinger Band, indicating a potential reversal from oversold conditions.

- **Sell Signal**: A sell order is executed when the price crosses above the upper Bollinger Band, signaling that the asset may be overbought and is likely to reverse downward.

- **Stop-Loss**: If the price falls by a specified percentage from the entry price (e.g., 5%), the position is closed to minimize losses.

### Example Logic:

1\. **Buy** when the price is below the lower Bollinger Band.

2\. **Sell** when the price is above the upper Bollinger Band or when the stop-loss is triggered (e.g., the price falls 5% below the buy price).


In [2]:
df = pd.DataFrame(yf.download("^NSEI", start="2014-07-01", end="2024-07-01"))

[*********************100%***********************]  1 of 1 completed


In [3]:
df.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-06-24,23382.300781,23558.099609,23350.000000,23537.849609,23537.849609,239400
2024-06-25,23577.099609,23754.150391,23562.050781,23721.300781,23721.300781,298100
2024-06-26,23723.099609,23889.900391,23670.449219,23868.800781,23868.800781,287800
2024-06-27,23881.550781,24087.449219,23805.400391,24044.500000,24044.500000,515200
2024-06-28,24085.900391,24174.000000,23985.800781,24010.599609,24010.599609,354800


In [4]:
df.isnull().sum().any()

False

In [127]:
def BollingerBands(data, period=20, multiplier=2):
    close = data.Close
    ma = np.convolve(close, np.ones(period)/period, mode='valid')  # Moving average
    std = np.array([np.std(close[i-period:i]) for i in range(period, len(close)+1)])  # Std dev
    upper_band = ma + (multiplier * std)
    lower_band = ma - (multiplier * std)

    # Padding the result to match the length of the original data
    pad_length = len(close) - len(ma)
    upper_band = np.concatenate([np.full(pad_length, np.nan), upper_band])
    lower_band = np.concatenate([np.full(pad_length, np.nan), lower_band])
    
    return upper_band, lower_band

In [131]:
class BollingerBandsStrategy(Strategy):
    # Initialize the strategy, defining the indicator
    def init(self):
        self.upper_band, self.lower_band = self.I(BollingerBands, self.data, 20, 2)

    # Define the strategy logic
    def next(self):
        # Buy when the price crosses below the lower Bollinger Band
        if self.data.Close[-1] < self.lower_band[-1]:
            # do not buy if we already have a position(long)
            if not self.position:
                stop_loss_price = self.data.Close[-1] * 0.95
                self.buy(sl=stop_loss_price)
            
        # Sell when the price crosses above the upper Bollinger Band
        elif self.data.Close[-1] > self.upper_band[-1]:
            # close all long positions
            self.position.close()
            

In [132]:
bt = Backtest(df, BollingerBandsStrategy, cash=100000, commission=0.002)

In [133]:
bt.run()

Start                     2015-01-02 00:00:00
End                       2024-09-27 00:00:00
Duration                   3556 days 00:00:00
Exposure Time [%]                   47.161937
Equity Final [$]                167166.239897
Equity Peak [$]                 167166.239897
Return [%]                           67.16624
Buy & Hold Return [%]              211.823054
Return (Ann.) [%]                     5.55279
Volatility (Ann.) [%]               12.681344
Sharpe Ratio                         0.437871
Sortino Ratio                         0.66046
Calmar Ratio                         0.192843
Max. Drawdown [%]                  -28.794336
Avg. Drawdown [%]                   -2.587937
Max. Drawdown Duration      873 days 00:00:00
Avg. Drawdown Duration       58 days 00:00:00
# Trades                                   37
Win Rate [%]                        56.756757
Best Trade [%]                       25.64667
Worst Trade [%]                     -6.726391
Avg. Trade [%]                    

In [136]:
bt.plot(filename="./plots/BollingerBandsSimple(20,2)_NIFTY_50.png", plot_volume=False, plot_pl=True)

c:\Users\tejas\anaconda3\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\tejas\anaconda3\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\tejas\anaconda3\Lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
c:\Users\tejas\anaconda3\Lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p7530', ...)